Preprocesing Text menggunakan NLTK

Nicholaus Verdhy || A11.2020.12447

# Penjelasan Singkat mengenai NLTK

NLTK adalah kumpulan alat dan program yang digunakan untuk memproses dan menganalisis bahasa alami (NLP) dalam bahasa Inggris. Ini ditulis dalam bahasa pemrograman Python dan digunakan untuk tugas-tugas NLP yang melibatkan analisis teks dan statistik[1]

# Referensi

[1]	E. J. Rifano, A. C. Fauzan, A. Makhi, E. Nadya, Z. Nasikin, and F. N. Putra, “Text Summarization Menggunakan Library Natural Language Toolkit (NLTK) Berbasis Pemrograman Python,” Ilk. J. Comput. Sci. Appl. Inform., vol. 2, no. 1, Art. no. 1, Apr. 2020, doi: 10.28926/ilkomnika.v2i1.32.

